In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
import torch.utils.data as data_utils
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
train=pd.read_csv('../data/train.csv', index_col='ID_code')
test=pd.read_csv('../data/test.csv', index_col='ID_code')
submission=pd.read_csv('../data/sample_submission.csv')

In [3]:
x = train.loc[:,train.columns !='target']
print(x.shape)
x.head()

(200000, 200)


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
y = train['target']
print(y.shape)
y.head()

(200000,)


ID_code
train_0    0
train_1    0
train_2    0
train_3    0
train_4    0
Name: target, dtype: int64

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scaler = StandardScaler()

In [7]:
x=scaler.fit_transform(x)

In [8]:
x.mean(), x.std()

(-1.1152668122349497e-15, 1.0000000000000013)

In [9]:
y = np.array(y)

In [10]:
y = y.reshape(y.shape[0],1)

In [11]:
print(y.shape)
y

(200000, 1)


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_valid, y_train, y_valid = train_test_split(x,y,test_size=0.3, random_state=42, shuffle=True)

In [14]:
x_train.shape, x_valid.shape,y_train.shape, y_valid.shape

((140000, 200), (60000, 200), (140000, 1), (60000, 1))

In [15]:
x_train=torch.FloatTensor(x_train)
x_valid=torch.FloatTensor(x_valid)
y_train=torch.FloatTensor(y_train)
y_valid=torch.FloatTensor(y_valid)

In [16]:
x_train.size(), x_valid.size(),y_train.size(), y_valid.size()

(torch.Size([140000, 200]),
 torch.Size([60000, 200]),
 torch.Size([140000, 1]),
 torch.Size([60000, 1]))

In [17]:
train = data_utils.TensorDataset(x_train, y_train)
valid = data_utils.TensorDataset(x_valid, y_valid)

In [18]:
trainloader = torch.utils.data.DataLoader(train, batch_size=20000, shuffle=True)
validloader = torch.utils.data.DataLoader(valid, batch_size=20000, shuffle=True)

In [19]:
train, label =next(iter(trainloader))

In [20]:
train.size(), label.size()

(torch.Size([20000, 200]), torch.Size([20000, 1]))

In [21]:
valid, label =next(iter(validloader))

In [22]:
valid.size(), label.size()

(torch.Size([20000, 200]), torch.Size([20000, 1]))

In [23]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(200, 150)
        self.fc2 = nn.Linear(150, 100)
        self.fc3 = nn.Linear(100, 50)
        self.fc4 = nn.Linear(50, 1)
        
        self.ac = nn.ReLU()
        self.dr = nn.Dropout(p=0.2)
        self.ac2 = nn.Sigmoid()
        
    def forward(self,x):
        x = self.dr(self.ac(self.fc1(x)))
        x = self.dr(self.ac(self.fc2(x)))
        x = self.dr(self.ac(self.fc3(x)))
        x = self.ac2(self.fc4(x))
                    
        return x
        
model=Model()

model

Model(
  (fc1): Linear(in_features=200, out_features=150, bias=True)
  (fc2): Linear(in_features=150, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=1, bias=True)
  (ac): ReLU()
  (dr): Dropout(p=0.2)
  (ac2): Sigmoid()
)

In [53]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [54]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [66]:
epochs = 50

train_loss =[]
valid_loss =[]
roc_auc_scores =[]


for e in range(epochs):
   
    running_losses = 0
    for trains, labels in trainloader:
        
        optimizer.zero_grad()
        
        output = model(trains)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_losses += loss.item()
        
    else:
        test_losses = 0
        auc = 0
        
        with torch.no_grad():
            model.eval()
            for valids, labels in validloader:
                
                ps = model(valids)
                test_losses += criterion(ps, labels).item()
                
                labels = labels.data.numpy()
                labels = labels.astype('int32')
                
                ps = ps.data.numpy()
                
                auc += roc_auc_score(labels, ps)
        
        train_loss.append(running_losses)
        valid_loss.append(test_losses)
        roc_auc_scores.append(roc_auc / (len(test)/len(labels)))
            

        model.train()
        
        if e % 10 == 0:
            print(f"epoch: {e}/{epochs}", 
                 f"train_loss: {train_loss[e]: .3f}",
                 f"valid_loss: {valid_loss[e]: .3f}",
                  f"roc_auc_score: {roc_auc_scores[e]: .3f}")

epoch: 0/50 train_loss:  1.075 valid_loss:  0.977 valid_loss:  0.254
epoch: 10/50 train_loss:  1.293 valid_loss:  0.834 valid_loss:  0.254
epoch: 20/50 train_loss:  1.206 valid_loss:  0.863 valid_loss:  0.254
epoch: 30/50 train_loss:  1.160 valid_loss:  0.888 valid_loss:  0.254
epoch: 40/50 train_loss:  1.101 valid_loss:  0.917 valid_loss:  0.254


In [ ]:
plt.plot(training_loss, label="training loss")
plt.plot(validation_loss, label="validation loss")
plt.legend()

## Inference

Now that the model is trained, we can use it for inference. We've done this before, but now we need to remember to set the model in inference mode with `model.eval()`. You'll also want to turn off autograd with the `torch.no_grad()` context.

In [ ]:
test = np.array(test)

In [ ]:
test=scaler.fit_transform(test)

In [ ]:
test.mean(), test.std()

In [ ]:
model.eval()


with torch.no_grad():
    output = model(test)